<a href="https://colab.research.google.com/github/RaiYan163/Transformers_LLM_prac/blob/main/transformerPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn as nn
import math

class InputEmbeddings(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)





In [23]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    #Creating a matrix of shape (seq_len, d_model)

    pe = torch.zeros(seq_len, d_model)

    #Creating a vector of shape (seq_len, 1)
    position = torch.arange(0, seq_len, dtype = torch.float).unsqueeze(1) # Did not understand the purpose of unsqueeze, it is said that it adds a dimension , but how and why?
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))

    #Applying sin to the even position
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)


    pe = pe.unsqueeze(0) # (1, Seq_len, d_model)
    self.register_buffer('pe', pe) #why we used it? Clear it out

    def forward(self, x):
      x = x + (self.pe[:, :x.shape[1], :]).requires_grad(False) # I am having problems regarding the dimensions
      return self.dropout(x) #please clear out the drop out






In [24]:
class LayerNormalization(nn.Module):
  def __init__(self, eps: float = 10**-6) -> None:
    super().__init__()
    self.eps = eps #Required for numerical stability
    self.alpha = nn.Parameter(torch.ones(1)) #multipliedm, it is said this makes the param learnable.
    self.bias = nn.Parameter(torch.zeros(1)) #added, it is single tensor (1, )

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim = True) # keeping the single mean in last dimensiion
    std = x.std(dim = -1, keepdim = True)
    return self.alpha * (x-mean) / (std + self.eps) + self.bias #bias and alpha parameter, whole ans er sathe added



In [25]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int, dropout: float)-> None:
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff) # W1 and B1
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model) # w2 and B2

  def forward(self, x):
    # (Batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

#why dropout, how and why linear transformation? How relu helping, and what is the purpose of FeedForwardBlock

In [26]:
class MultiHeadAttentionBlock(nn.Module):

  def __init__(self, d_model: int, h: int, dropout: float)-> None:
    super().__init__()
    self.d_model = d_model
    self.h = h
    assert d_model % h == 0, "d_model is not divisble by h"

    self.d_k = d_model // h
    self.w_q = nn.Linear(d_model, d_model) #Wq
    self.w_k = nn.Linear(d_model, d_model) #Wk
    self.w_v = nn.Linear(d_model, d_model) #Wv

    self.w_o = nn.Linear(d_model, d_model) #W0
    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k = query.shape[-1]

    # (Batch, h, Seq_len, d_k) --> (batch, h, seq_len, seq_len)
    attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
      attention_scores.masked_fill(mask == 0, -1e9)
    attention_scores = attention_scores.softmax(dim = -1) # (Batch, h, seq_len, seq_len)
    if dropout is not None:
      attention_scores = dropout(attention_score)

    return (attention_scores @ value), attention_scores


  def forward(self, q, k, v, mask):
    query = self.w_q(q) # (Batch, Seq_Len, d_model) ---> (Batch, Seq_len, d_model)
    key = self.w_k(k) # (Batch, Seq_Len, d_model) ---> (Batch, Seq_len, d_model)
    value = self.w_v(v) # (Batch, Seq_Len, d_model) ---> (Batch, Seq_len, d_model)

    # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
    key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
    # Why the transpose was needed, what is it's benifit?

    x, self.attention_score = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

    # (Batch, h, Seq_len, d_k) --> (Batch, Seq_len, h, d_k) --> (Batch, Seq_len, d_model)
    x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

    #(Batch, seq_len, d_model) --> (Batch, Seq_len, d_model)
    return self.w_o(x)


# Need to understand the remaining, please review later

In [27]:
class ResidualConnection(nn.Module):

  def __init__(self, dropout: float) -> None:
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = LayerNormalization()

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x))) # where did this sublayer come from? What is it's purpose?








In [28]:
class EncoderBlock(nn.Module):

  def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)]) #storing two residual connection here

  def forward(self, x, src_mask):
    x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, src_mask)) #confused how residual connections are taking input
    x = self.residual_connections[1](x, self.feed_forward_block)
    return x


In [29]:
class Encoder(nn.Module):
  def __init__(self, layers: nn.ModuleList) -> None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [30]:
class ProjectionLayer(nn.Module):

  def __init__(self, d_model: int, vocab_size: int) -> None:
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    #(Batch, Seq_len, d_model) ---> (Batch, seq_len, Vocab_size)
    return torch.log_softmax(self.proj(x), dim = -1)

In [31]:
class DecoderBlock(nn.Module):

  def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connection = nn.Module([ResidualConnection(dropout) for _ in range(3)]) # 3 residual connection used here, didn't get the purpose of dropout in this case.

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
    x = self.residual_connection[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connection[2](x, self.feed_forward_block)
    return x



In [32]:
class Decoder(nn.Module):
  def __init__(self, layers: nn.ModuleList) -> None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, encoder_output, src_mask, tgt_mask):
      for layer in self.layers:
        x = layer(x, encoder_output, src_mask, tgt_mask)
      return self.norm(x)

# `nn.ModuleList` is a container in PyTorch to store a list of `nn.Module` instances,
# typically layers, within a neural network model. It allows for easy management,
# parameter tracking, and iteration over the layers, facilitating sequential
# application of operations during the forward pass.


In [33]:
class Transformer(nn.Module):

  def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    return self.projection_layer(x)

def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048)->Transformer:
  # Create the embedding layer

  src_embed = InputEmbeddings(d_model, src_vocab_size)
  tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)


  # Create the positional encoding layer
  src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
  tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

  decoder_blocks = []

  for _ in range(N):
    decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
    decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
    decoder_blocks.append(decoder_block)

  # creating the encoder and decoder
  projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

  #Creating the transformer
  transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

  #Initialize the paramters
  for p in transformer.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  return transformer

# Notice very carefully, build transformer is another defined function here




##Tokenizer (Word Level)


In [34]:
pip install datasets

In [35]:
import torch
import torch.nn as nn

#import from hugging face

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split

In [36]:
def get_all_sentences(ds, lang):
  for item in ds:
    yield item['translation'][lang]


def get_or_build_tokenizer(config, ds, lang):
  # config['tokenizer_file']
  tokenizer_path = Path(config['tokenizer_file'].format(lang)) # Language will determine whether it is in english, or italian, or define any language.
  if not Path.exists(tokenizer_path):
    tokenizer = Tokenizer(WordLevel(unk_token ='[UNK]'))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
    tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
    tokenizer.save(str(tokenizer_path))
  else:
    tokenizer = Tokenizer.from_file(str(tokenizer_path))

  return tokenizer


  def get_ds(config):
    ds_raw = load_dataset('opus_books', f'{config["lang-src"]}-{config["lang_tgt"]}', split='train')

    #Build Tokenizer
    tokenizer_src = get_or_build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config['lang_tgt'])

    #keeping 90% for training and 10% for validation
    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])